<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/SBP-Brims/Notebooks/Cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/"

## Imports

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=ef6b64b890470e2680463ba75457a02934069a33f713c4aae032595eb1fbe75e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


### Function

In [ ]:
# Stop words and empty lines
# (PNA photo by
# (Photo courtesy by ) not captured
import re

def make_annotateDF(df, isSunStar=False):
    workingDF = pd.DataFrame(columns=['Annotate','Text'])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])

        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index]}, index=[0])
            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
            text = str(df['Text'].iloc[index])
            # print(text)
            if isSunStar:
                sentences = sent_tokenize(text)
            else:
                sentences = text.split('\n')

            for sentence in sentences:
                sentence = re.sub(r"http\S+", "", sentence)
                sentence = re.sub(r"@\S+", "", sentence)
                sentence = re.sub(r"\S+.com\S+", "", sentence)
                sentence = sentence.replace("– Rappler.com","")
                sentence = sentence.replace("(PNA)","")

                if (', GMA Integrated News' in sentence
                    or sentence ==''
                    or sentence =='– Rappler.com'
                    or 'SunStar Philippines)' in sentence
                    or 'File photo' in sentence
                    or 'Courtesy of ' in sentence
                    or len(sentence.split()) <3
                    or 'RELATED STORY' in sentence
                    or '(Photo: '  in sentence
                    or 'Rappler.com' in sentence
                    or '(Photo courtesy of' in sentence
                    ):
                    # print(sentence)
                    pass
                else:
                    new_row = pd.DataFrame({'Annotate':'', 'Text': sentence}, index=[0])
                    workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)

    return workingDF


In [ ]:
from langdetect import detect
import re

from langdetect import DetectorFactory
DetectorFactory.seed = 0

def get_headlineLead(df, isSunStar=False, isPNA=False):
    workingDF = pd.DataFrame(columns=['Text', 'Annotate'])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])
        try:
            lang = detect(df['Title'].iloc[index])
        except:
            lang = ""

        if (lang == 'tl'):
            # print(df['Title'].iloc[index])
            pass
        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index]}, index=[0])
            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
            text = str(df['Text'].iloc[index])

            if isSunStar:
                text = re.sub(r'\.(?=[^ \W\d])', '. ', text)
                sentences = sent_tokenize(text)

            else:
                sentences = text.split('\n')

            if isPNA:
                for text in sentences:
                    if text.startswith('MANILA'):
                        sentence = text
                        sentence = sentence.strip()
                        break
            else:
                sentence = sentences[0].strip()


            sentence = re.sub(r"http\S+", "", sentence)
            sentence = re.sub(r"@\S+", "", sentence)
            sentence = re.sub(r"\S+.com\S+", "", sentence)
            sentence = sentence.replace("– Rappler.com","")
            sentence = sentence.replace("(PNA)","")

            if (', GMA Integrated News' in sentence
                    or sentence ==''
                    or sentence =='– Rappler.com'
                    or 'SunStar Philippines)' in sentence
                    or 'File photo' in sentence
                    or 'Courtesy of ' in sentence
                    or len(sentence.split()) <3
                    or 'RELATED STORY' in sentence
                    or '(Photo: '  in sentence
                    or 'Rappler.com' in sentence
                    or '(Photo courtesy of' in sentence
                    ):
                    # print(sentence)
                pass
            else:
                new_row = pd.DataFrame({'Annotate':'', 'Text': sentence}, index=[0])
                workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)

    return workingDF


### SBP

In [ ]:
from langdetect import detect
import re

from langdetect import DetectorFactory
DetectorFactory.seed = 0

def getSpecificArticle(df, isSunStar=False, keywords=[):
    workingDF = pd.DataFrame(columns=['Text', 'Annotate'])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])
        try:
            lang = detect(df['Title'].iloc[index])
        except:
            lang = ""

        if (lang == 'tl'):
            # print(df['Title'].iloc[index])
            pass
        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            for key in keywords:
                if(key in row['Title'] or key in str(df['Text'].iloc[index])):
                    new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index]}, index=[0])
                    workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
                    text = str(df['Text'].iloc[index])

                    if isSunStar:
                        text = re.sub(r'\.(?=[^ \W\d])', '. ', text)
                        sentences = sent_tokenize(text)

                    else:
                        sentences = text.split('\n')

                    for sentence in sentences:
                        sentence = re.sub(r"http\S+", "", sentence)
                        sentence = re.sub(r"@\S+", "", sentence)
                        sentence = re.sub(r"\S+.com\S+", "", sentence)
                        sentence = sentence.replace("– Rappler.com","")
                        sentence = sentence.replace("(PNA)","")

                        if (', GMA Integrated News' in sentence
                            or sentence ==''
                            or sentence =='– Rappler.com'
                            or 'SunStar Philippines)' in sentence
                            or 'File photo' in sentence
                            or 'Courtesy of ' in sentence
                            or len(sentence.split()) <3
                            or 'RELATED STORY' in sentence
                            or '(Photo: '  in sentence
                            or 'Rappler.com' in sentence
                            or '(Photo courtesy of' in sentence
                            ):
                            # print(sentence)
                            pass
                        else:
                            new_row = pd.DataFrame({'Annotate':'', 'Text': sentence}, index=[0])
                            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)



    return workingDF

In [ ]:
from langdetect import detect
import re

from langdetect import DetectorFactory
DetectorFactory.seed = 0

def getSpecificArticle_time(df, isSunStar=False, keywords=[], outlet):
    workingDF = pd.DataFrame(columns=['Text', 'Annotate', "Outlet", "Date"])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])
        try:
            lang = detect(df['Title'].iloc[index])
        except:
            lang = ""

        if (lang == 'tl'):
            # print(df['Title'].iloc[index])
            pass
        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            df['Date'] = pd.to_datetime(df['Date'])
            for key in keywords:
                if(key in row['Title'] or key in str(df['Text'].iloc[index])):
                    new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index], 'Outlet': outlet, "Date": df['Date'].iloc[index] }, index=[0])
                    workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
                    text = str(df['Text'].iloc[index])

                    if isSunStar:
                        text = re.sub(r'\.(?=[^ \W\d])', '. ', text)
                        sentences = sent_tokenize(text)

                    else:
                        sentences = text.split('\n')

                    for sentence in sentences:
                        sentence = re.sub(r"http\S+", "", sentence)
                        sentence = re.sub(r"@\S+", "", sentence)
                        sentence = re.sub(r"\S+.com\S+", "", sentence)
                        sentence = sentence.replace("– Rappler.com","")
                        sentence = sentence.replace("(PNA)","")

                        if (', GMA Integrated News' in sentence
                            or sentence ==''
                            or sentence =='– Rappler.com'
                            or 'SunStar Philippines)' in sentence
                            or 'File photo' in sentence
                            or 'Courtesy of ' in sentence
                            or len(sentence.split()) <3
                            or 'RELATED STORY' in sentence
                            or '(Photo: '  in sentence
                            or 'Rappler.com' in sentence
                            or '(Photo courtesy of' in sentence
                            ):
                            # print(sentence)
                            pass
                        else:
                            new_row = pd.DataFrame({'Annotate':'', 'Text': sentence, , 'Outlet': outlet, "Date": df['Date'].iloc[index]}, index=[0])
                            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)



    return workingDF

## Main

### GMA

In [ ]:
gma = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/GMA_2023-05-07NEWS.csv')
gma

,Unnamed: 0,Title,Author,Date,Text
0,0,"DOH logs 1,920 new COVID-19 cases, highest dai...",By GMA Integrated News,2023-05-07,"The Philippines on Sunday registered 1,920 ne..."
1,1,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN,2023-05-07,The Department of Health said Saturday that t...
2,2,Bettor wins P55.3 million jackpot in Grand Lot...,By GMA Integrated News,2023-05-07,"A bettor won the jackpot prize of P55,394,255..."
3,3,Filipinos watch as King Charles III of Britain...,By GMA Integrated News,2023-05-06,Filipinos monitored the coronation of the Uni...
4,4,"Marcos will not abolish NTF-ELCAC, says NSC",By GMA Integrated News,2023-05-06,President Ferdinand R. Marcos Jr. has no inte...
...,...,...,...,...,...
2218,2218,"SC upholds House journal over videos, photos; ...","By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Supreme Court has upheld the constitution...
2219,2219,Philippines' justice chief says ICC drug war p...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,Justice Secretary Jesus Crispin Remulla descr...
2220,2220,De Lima drug cases may be resolved by March —l...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The camp of detained former Senator Leila de ...
2221,2221,Philippines to appeal ICC’s resumption of drug...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Philippine government intends to appeal t...


In [ ]:
gma_df = make_annotateDF(gma)
gma_df

,Annotate,Text
0,,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,,"The Philippines on Sunday registered 1,920 ne..."
2,,"According to GMA Integrated News Research, th..."
3,,"It is the fourth straight day of more than 1,..."
4,,Data from the Department of Health's COVID-19...
...,...,...
27781,,"A total of 1,964,679 individuals or 481,012 f..."
27782,,"Of the affected population, 88,134 individual..."
27783,,"A total of 1,858 houses were reported damaged..."
27784,,"Damage to agriculture worth PP885,165,517 and..."


In [ ]:
gma_df.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-full.csv', index=False)

In [ ]:
gmaHL_df = get_headlineLead(gma)
gmaHL_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",
1,"The Philippines on Sunday registered 1,920 new...",
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",
3,The Department of Health said Saturday that th...,
4,Bettor wins P55.3 million jackpot in Grand Lot...,
...,...,...
4406,The camp of detained former Senator Leila de L...,
4407,Philippines to appeal ICC’s resumption of drug...,
4408,The Philippine government intends to appeal th...,
4409,Reported death toll from bad weather now at 40...,


In [ ]:
gmaHL_df.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-HL.csv', index=False)

In [ ]:
gmaHL_label = gmaHL_df
gmaHL_label['Media_Outlet'] = ['GMA']*4411
gmaHL_label

,Text,Annotate,Media_Outlet
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",,GMA
1,"The Philippines on Sunday registered 1,920 new...",,GMA
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",,GMA
3,The Department of Health said Saturday that th...,,GMA
4,Bettor wins P55.3 million jackpot in Grand Lot...,,GMA
...,...,...,...
4406,The camp of detained former Senator Leila de L...,,GMA
4407,Philippines to appeal ICC’s resumption of drug...,,GMA
4408,The Philippine government intends to appeal th...,,GMA
4409,Reported death toll from bad weather now at 40...,,GMA


### PNA

In [ ]:
pna = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/PhilippineNewsAgency_2023-05-06NEWS.csv')
pna

,Unnamed: 0,Title,Author,Date,Text
0,0,"MIAA, Meralco begins NAIA T3 electrical audit",NaN,"May 5, 2023",File photo\nMANILA – The Manila International ...
1,1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",NaN,"May 5, 2023","MANILA – Marilene Acosta, the CEO of Pag-IBIG ..."
2,2,DMW assures aid for OFWS from Sudan who wanted...,By Marita Moaje,"May 5, 2023",HOME. A total of 74 repatriates from war-torn ...
3,3,PNP chief fetes HPG's vital role in upholding ...,By Christopher Lloyd Caliwan,"May 5, 2023",PNP chief Gen. Benjamin Acorda Jr. (PNA photo ...
4,4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",By Raymond Carl Dela Cruz,"May 5, 2023",MANILA – The chief of the Land Transportation ...
...,...,...,...,...,...
2195,2195,PBBM's 'Pambansang Pabahay’ empowers LGUs to a...,By Zaldy De Layola,"February 8, 2023",(File photo)\nMANILA – The Department of Human...
2196,2196,1.2K cops to secure ASEAN education execs' mee...,By Christopher Lloyd Caliwan,"February 8, 2023",(Photo courtesy of NCRPO)\nMANILA – More than ...
2197,2197,PNP reaffirms support to Interpol efforts vs. ...,By Christopher Lloyd Caliwan,"February 8, 2023",Interpol Secretary General Jürgen Stock (left)...
2198,2198,"IP leaders call for unity to preserve IPRA, NC...","By Perfecto Raymundo, Jr. and Che Palicte","February 8, 2023",UNITY. Indigenous people leaders call for unit...


In [ ]:
pna_df = make_annotateDF(pna)
pna_df

,Annotate,Text
0,,"MIAA, Meralco begins NAIA T3 electrical audit"
1,,MANILA – The Manila International Airport Auth...
2,,MIAA officer-in-charge Bryan Co said the audit...
3,,"Co added that once the audit is concluded, the..."
4,,The MIAA will then determine the timeline as w...
...,...,...
28692,,He added that Philippine Rear Admiral Renato ...
28693,,"""The meeting will yield meaningful discussions..."
28694,,He added the staff-to-staff talks between the ...
28695,,Badidles added that the activity is part of th...


In [ ]:
pna_df.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-full.csv', index=False)

In [ ]:
pnaHL_df = get_headlineLead(pna, isPNA=True)
pnaHL_df

,Text,Annotate
0,"MIAA, Meralco begins NAIA T3 electrical audit",
1,MANILA – The Manila International Airport Auth...,
2,"Pag-IBIG, Singaporean firm eye building ‘net-z...",
3,"MANILA – Marilene Acosta, the CEO of Pag-IBIG ...",
4,DMW assures aid for OFWS from Sudan who wanted...,
...,...,...
4395,"MANILA – More than 1,200 police officers have ...",
4396,"IP leaders call for unity to preserve IPRA, NC...",
4397,MANILA – Leaders of the indigenous peoples (IP...,
4398,"PH, US Navy fleets to tackle cooperation, exch...",


In [ ]:
pnaHL_df.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-HL.csv', index=False)

In [ ]:
pnaHL_label = pnaHL_df
pnaHL_label['Media_Outlet'] = ['Philippine News Agency']*4400
pnaHL_label

,Text,Annotate,Media_Outlet
0,"MIAA, Meralco begins NAIA T3 electrical audit",,Philippine News Agency
1,MANILA – The Manila International Airport Auth...,,Philippine News Agency
2,"Pag-IBIG, Singaporean firm eye building ‘net-z...",,Philippine News Agency
3,"MANILA – Marilene Acosta, the CEO of Pag-IBIG ...",,Philippine News Agency
4,DMW assures aid for OFWS from Sudan who wanted...,,Philippine News Agency
...,...,...,...
4395,"MANILA – More than 1,200 police officers have ...",,Philippine News Agency
4396,"IP leaders call for unity to preserve IPRA, NC...",,Philippine News Agency
4397,MANILA – Leaders of the indigenous peoples (IP...,,Philippine News Agency
4398,"PH, US Navy fleets to tackle cooperation, exch...",,Philippine News Agency


### Rappler

In [ ]:
rappler = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/Rappler_2023-05-07NEWS.csv')
rappler

,Unnamed: 0,Title,Author,Date,Text
0,0,Marcos the party-going schoolboy returns to UK...,Lian Buan,2023-05-05,"LONDON, United Kingdom – In the late ’70s, Fer..."
1,1,"Ablan confirms resignation from DepEd, denies ...",Ryan Macasero,2023-05-05,"MANILA, Philippines – Former Department of Edu..."
2,2,WATCH: Paano naibenta sa labas ang DepEd lapto...,Rappler.com,2023-05-05,"MANILA, Philippines – Naaalala ‘nyo ba ‘yung m..."
3,3,"In unprecedented move, Filipino journalist har...",Gaby Baizas,2023-05-05,"MANILA, Philippines – Days after media workers..."
4,4,Marcos says Philippines bases could be ‘useful...,Reuters,2023-05-05,"WASHINGTON DC, USA – Philippines President Fer..."
...,...,...,...,...,...
2196,2196,Fewer Filipinos skeptical of COVID-19 jabs – SWS,Sofia Tomacruz,2022-01-21,"MANILA, Philippines – As the coronavirus is po..."
2197,2197,"Duterte appoints Malacañang official, ex-Napol...",NaN,2022-01-21,"MANILA, Philippines – President Rodrigo Dutert..."
2198,2198,‘Ironclad’ PH-South Korea alliance seen with s...,Jairo Bolledo,2022-01-21,"MANILA, Philippines – The Philippines is looki..."
2199,2199,"3 Cordillera provinces, Northern Samar placed ...",NaN,2022-01-20,"MANILA, Philippines – The alert level of three..."


In [ ]:
rappler_df = make_annotateDF(rappler)
rappler_df

,Annotate,Text
0,,Marcos the party-going schoolboy returns to UK...
1,,"LONDON, United Kingdom – In the late ’70s, Fer..."
2,,"Marcos, his first lady Louise Araneta Marcos a..."
3,,"“Sa palagay ko, dahil magkakilala kami, eh dap..."
4,,"Besides, said Locsin, “he left his heart here.”"
...,...,...
33998,,"On January 22, Jimenez acknowledged that the T..."
33999,,“Because the ballots have not yet been printed...
34000,,The printing of ballots was supposed to start ...
34001,,Some 67.4 million ballots will be printed over...


In [ ]:
rappler_df.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-full.csv', index=False)

In [ ]:
rapplerHL_df = get_headlineLead(rappler)
rapplerHL_df

,Text,Annotate
0,Marcos the party-going schoolboy returns to UK...,
1,"LONDON, United Kingdom – In the late ’70s, Fer...",
2,"Ablan confirms resignation from DepEd, denies ...",
3,"MANILA, Philippines – Former Department of Edu...",
4,WATCH: Paano naibenta sa labas ang DepEd lapto...,
...,...,...
4394,"MANILA, Philippines – The Philippines is looki...",
4395,"3 Cordillera provinces, Northern Samar placed ...",
4396,"MANILA, Philippines – The alert level of three...",
4397,Comelec starts printing ballots for 2022 polls,


In [ ]:
rapplerHL_df.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-HL.csv', index=False)

In [ ]:
rapplerHL_label = rapplerHL_df
rapplerHL_label['Media_Outlet'] = ['Rappler']*4399
rapplerHL_label

,Text,Annotate,Media_Outlet
0,Marcos the party-going schoolboy returns to UK...,,Rappler
1,"LONDON, United Kingdom – In the late ’70s, Fer...",,Rappler
2,"Ablan confirms resignation from DepEd, denies ...",,Rappler
3,"MANILA, Philippines – Former Department of Edu...",,Rappler
4,WATCH: Paano naibenta sa labas ang DepEd lapto...,,Rappler
...,...,...,...
4394,"MANILA, Philippines – The Philippines is looki...",,Rappler
4395,"3 Cordillera provinces, Northern Samar placed ...",,Rappler
4396,"MANILA, Philippines – The alert level of three...",,Rappler
4397,Comelec starts printing ballots for 2022 polls,,Rappler


### SunStar

In [ ]:
sunstar = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/SunStar_2023-05-07NEWS.csv')
sunstar

,Unnamed: 0,Title,Author,Date,Text
0,0,Sandiganbayan affirms denial of motion seeking...,Third Anne Peralta-Malonzo,"May 07, 2023",THE Sandiganbayan has affirmed the decision of...
1,1,PH to reassess protocols with end of Covid glo...,NaN,"May 06, 2023",WITH the end of the global emergency status fo...
2,2,DMW: Work experience top criterion for oversea...,NaN,"May 06, 2023",THE Department of Migrant Workers (DMW) on Sat...
3,3,Marcos accepts courtesy resignation of 2 senio...,Third Anne Peralta-Malonzo,"May 05, 2023",PRESIDENT Ferdinand “Bongbong” Marcos has conf...
4,4,"DOH reiterates call for kids to get measles, p...",NaN,"May 05, 2023",FIVE days after starting its mass immunization...
...,...,...,...,...,...
2261,2261,"CSF indigents get cash aid, food packs from Guv",Ian Ocampo Flora,"January 30, 2023",CITY OF SAN FERNANDO The provincial government...
2262,2262,AC execs help Bale Pusu ward reunite with family,Charlene A. Cayabyab,"January 28, 2023",ANGELES CITY The city government here has faci...
2263,2263,Cops seize P14-M 'shabu' in Bulacan,Princess Clea Arcellaz,"January 28, 2023",CITY OF SAN FERNANDO More than two kilos of su...
2264,2264,Crissot's 156 birth anniversary commemorated,Ian Ocampo Flora,"January 28, 2023",BACOLOR The heirs of Juan Crisostomo Soto (Cri...


In [ ]:
sunstar_df = make_annotateDF(sunstar, isSunStar=True)
sunstar_df

,Annotate,Text
0,,Sandiganbayan affirms denial of motion seeking...
1,,THE Sandiganbayan has affirmed the decision of...
2,,A careful reading of the defendants’ motion sh...
3,,"In view of the foregoing, the defendants faile..."
4,,The Court thus maintains its earlier pronounce...
...,...,...
17797,,The said session was attended by representativ...
17798,,"During the inquiry, SFELAPCO representatives e..."
17799,,It was also disclosed that the expiration of S...
17800,,Macalino assailed SFELAPCO’s alleged lack of c...


In [ ]:
sunstar_df.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-full.csv', index=False)

In [ ]:
sunstarHL_df = get_headlineLead(sunstar, isSunStar=True)
sunstarHL_df

,Text,Annotate
0,Sandiganbayan affirms denial of motion seeking...,
1,THE Sandiganbayan has affirmed the decision of...,
2,PH to reassess protocols with end of Covid glo...,
3,WITH the end of the global emergency status fo...,
4,DMW: Work experience top criterion for oversea...,
...,...,...
4516,CITY OF SAN FERNANDO More than two kilos of su...,
4517,Crissot's 156 birth anniversary commemorated,
4518,BACOLOR The heirs of Juan Crisostomo Soto (Cri...,
4519,CSF council inquiry on power rate hike launched,


In [ ]:
sunstarHL_df.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-HL.csv', index=False)

In [ ]:
sunstarHL_label = sunstarHL_df
sunstarHL_label['Media_Outlet'] = ['Sunstar']*4521
sunstarHL_label

,Text,Annotate,Media_Outlet
0,Sandiganbayan affirms denial of motion seeking...,,Sunstar
1,THE Sandiganbayan has affirmed the decision of...,,Sunstar
2,PH to reassess protocols with end of Covid glo...,,Sunstar
3,WITH the end of the global emergency status fo...,,Sunstar
4,DMW: Work experience top criterion for oversea...,,Sunstar
...,...,...,...
4516,CITY OF SAN FERNANDO More than two kilos of su...,,Sunstar
4517,Crissot's 156 birth anniversary commemorated,,Sunstar
4518,BACOLOR The heirs of Juan Crisostomo Soto (Cri...,,Sunstar
4519,CSF council inquiry on power rate hike launched,,Sunstar


### Manila Bulletin

In [ ]:
mb = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/manilaBulletin_2023-05-05_NEWS.csv')
mb

,Unnamed: 0,Title,Author,Date,Text
0,0,'Thank you gov': Cong Roman mourns passing of ...,Ellson Quismorio,"May 5, 2023",Bataan 1st district Rep. Geraldine Roman paid ...
1,1,P5-M reward offered to find missing Chinese pa...,Martin Sadongdong,"May 5, 2023",A P5-million reward was offered to anyone who ...
2,2,"Hontiveros: Nuclear energy a terrible, false s...",Hannah Torregoza,"May 5, 2023","\nSenator Risa Hontiveros on Friday, May 5 qu..."
3,3,You go girl! Pro-sports solon hails Kaila Napo...,Ellson Quismorio,"May 5, 2023",The first one is always special.\nPuwersa ng B...
4,4,Expedite release of Excellence in Teacher Educ...,Merlina Hernando-Malipot,"May 5, 2023",To further strengthen teacher education in the...
...,...,...,...,...,...
1999,1999,"Diokno, Australian envoy share notes on deepen...",Raymund Antonio,"Mar 25, 2023",Finance Secretary Benjamin Diokno met with Aus...
2000,2000,Australian gov't drums up interest in PH airpo...,Raymund Antonio,"Mar 25, 2023",The Australian government is looking to “drum ...
2001,2001,Proposed EBET law passed on 3rd reading; here'...,Ellson Quismorio,"Mar 25, 2023",The lower chamber recently gave its final appr...
2002,2002,Immediate expansion of gov’t subsidies to kind...,Merlina Hernando-Malipot,"Mar 25, 2023",A network of advocates called for the immediat...


In [ ]:
mb_df = make_annotateDF(mb)
mb_df

,Annotate,Text
0,,'Thank you gov': Cong Roman mourns passing of ...
1,,Bataan 1st district Rep. Geraldine Roman paid ...
2,,"""It is with a heavy heart that I offer my sinc..."
3,,"""His to enriching the lives of others through..."
4,,"""Thank you Governor Padilla for your service. ..."
...,...,...
23804,,"""The Philippines and China also discussed new ..."
23805,,"On Thursday, Lazaro started chairing with Chin..."
23806,,"The conduct of the BCM, a confidence-building ..."
23807,,"In an earlier pronouncement, Lazaro asserted t..."


In [ ]:
mb_df.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-full.csv', index=False)

In [ ]:
mbHL_df = get_headlineLead(mb)
mbHL_df

,Text,Annotate
0,'Thank you gov': Cong Roman mourns passing of ...,
1,Bataan 1st district Rep. Geraldine Roman paid ...,
2,P5-M reward offered to find missing Chinese pa...,
3,A P5-million reward was offered to anyone who ...,
4,"Hontiveros: Nuclear energy a terrible, false s...",
...,...,...
3868,The lower chamber recently gave its final appr...,
3869,Immediate expansion of gov’t subsidies to kind...,
3870,A network of advocates called for the immediat...,
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",


In [ ]:
mbHL_df.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-HL.csv', index=False)

In [ ]:
mbHL_label = mbHL_df
mbHL_label['Media_Outlet'] = ['Manila Bulletin']*3873
mbHL_label

,Text,Annotate,Media_Outlet
0,'Thank you gov': Cong Roman mourns passing of ...,,Manila Bulletin
1,Bataan 1st district Rep. Geraldine Roman paid ...,,Manila Bulletin
2,P5-M reward offered to find missing Chinese pa...,,Manila Bulletin
3,A P5-million reward was offered to anyone who ...,,Manila Bulletin
4,"Hontiveros: Nuclear energy a terrible, false s...",,Manila Bulletin
...,...,...,...
3868,The lower chamber recently gave its final appr...,,Manila Bulletin
3869,Immediate expansion of gov’t subsidies to kind...,,Manila Bulletin
3870,A network of advocates called for the immediat...,,Manila Bulletin
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",,Manila Bulletin


## END

In [ ]:
ALL_df = pd.concat([gma_df, pna_df, rappler_df, sunstar_df, mb_df])
ALL_df

# ALL_df.to_csv(f'{dir}/Official/curated_Dataset.csv', index=False)


In [ ]:
ALL_HL_df = pd.concat([gmaHL_df, pnaHL_df, rapplerHL_df, sunstarHL_df, mbHL_df])
ALL_HL_df

ALL_HL_df.to_csv(f'{dir}/Official/curated_Dataset_headlines_lead.csv', index=False)

In [ ]:
ALL_HL_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",
1,"The Philippines on Sunday registered 1,920 new...",
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",
3,The Department of Health said Saturday that th...,
4,Bettor wins P55.3 million jackpot in Grand Lot...,
...,...,...
3868,The lower chamber recently gave its final appr...,
3869,Immediate expansion of gov’t subsidies to kind...,
3870,A network of advocates called for the immediat...,
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",


In [ ]:
ALL_df

,Annotate,Text
0,,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,,"The Philippines on Sunday registered 1,920 ne..."
2,,"According to GMA Integrated News Research, th..."
3,,"It is the fourth straight day of more than 1,..."
4,,Data from the Department of Health's COVID-19...
...,...,...
23804,,"""The Philippines and China also discussed new ..."
23805,,"On Thursday, Lazaro started chairing with Chin..."
23806,,"The conduct of the BCM, a confidence-building ..."
23807,,"In an earlier pronouncement, Lazaro asserted t..."


In [ ]:
ALL_HL_label = pd.concat([gmaHL_label, pnaHL_label, rapplerHL_label, sunstarHL_df, mbHL_label])
ALL_HL_label



,Text,Annotate,Media_Outlet
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",,GMA
1,"The Philippines on Sunday registered 1,920 new...",,GMA
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",,GMA
3,The Department of Health said Saturday that th...,,GMA
4,Bettor wins P55.3 million jackpot in Grand Lot...,,GMA
...,...,...,...
3868,The lower chamber recently gave its final appr...,,Manila Bulletin
3869,Immediate expansion of gov’t subsidies to kind...,,Manila Bulletin
3870,A network of advocates called for the immediat...,,Manila Bulletin
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",,Manila Bulletin


In [ ]:
ALL_HL_label.to_csv(f'{dir}dataset/Extracted-News-May5-7/curated_Dataset_headlines_lead-MO.csv', index=False)


# SBP same issue

## GMA

In [ ]:
gma = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/GMA_2023-05-07NEWS.csv')
gma

,Unnamed: 0,Title,Author,Date,Text
0,0,"DOH logs 1,920 new COVID-19 cases, highest dai...",By GMA Integrated News,2023-05-07,"The Philippines on Sunday registered 1,920 ne..."
1,1,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN,2023-05-07,The Department of Health said Saturday that t...
2,2,Bettor wins P55.3 million jackpot in Grand Lot...,By GMA Integrated News,2023-05-07,"A bettor won the jackpot prize of P55,394,255..."
3,3,Filipinos watch as King Charles III of Britain...,By GMA Integrated News,2023-05-06,Filipinos monitored the coronation of the Uni...
4,4,"Marcos will not abolish NTF-ELCAC, says NSC",By GMA Integrated News,2023-05-06,President Ferdinand R. Marcos Jr. has no inte...
...,...,...,...,...,...
2218,2218,"SC upholds House journal over videos, photos; ...","By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Supreme Court has upheld the constitution...
2219,2219,Philippines' justice chief says ICC drug war p...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,Justice Secretary Jesus Crispin Remulla descr...
2220,2220,De Lima drug cases may be resolved by March —l...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The camp of detained former Senator Leila de ...
2221,2221,Philippines to appeal ICC’s resumption of drug...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Philippine government intends to appeal t...


In [ ]:
gma['Date'] = pd.to_datetime(gma['Date'])
gma_week =  gma.loc[gma['Date'] >= np.datetime64('2023-04-23')]
gma_week = gma_week.loc[ gma_week['Date'] <= np.datetime64('2023-04-29')]
gma_week = gma_week.reset_index()
gma_week = gma_week.drop(['Unnamed: 0', 'index'], axis=1)
gma_week

,Title,Author,Date,Text
0,DOH reports 304 new COVID-19 cases; active tal...,"By RICHA NORIEGA, GMA Integrated News",2023-04-29,The Philippines reported 304 new COVID-19 cas...
1,Some private schools ask for 4% tuition increa...,"By SHERYLIN UNTALAN, GMA Integrated News",2023-04-29,Members of the Coordinating Council of Privat...
2,AFP refuses to comment on PH-China vessels inc...,"By TED CORDERO, GMA Integrated News",2023-04-29,The Armed Forces of the Philippines has decli...
3,Ex-Ombudsman Gutierrez takes oath as GSIS acti...,By GMA Integrated News,2023-04-28,Former Ombudsman Merceditas Gutierrez took he...
4,"Bantag, Zulueta now on hold departure list — BI","By RICHA NORIEGA, GMA Integrated News",2023-04-28,Former Bureau of Corrections (BuCor) chief Ge...
...,...,...,...,...
131,Marcos to new PNP chief: Enforce maximum toler...,"By ANNA FELICIA BAJO, GMA Integrated News",2023-04-24,"President Ferdinand ""Bongbong"" Marcos Jr. on ..."
132,ACT calls on DepEd to add classroom facilities...,NaN,2023-04-23,The Alliance of Concerned Teachers (ACT) over...
133,DOTr: Face-mask wearing remains mandatory in L...,NaN,2023-04-23,The Department of Transportation said on Sund...
134,AFP says it's possible to expand EDCA sites fu...,"By GISELLE OMBAY, GMA Integrated News",2023-04-23,The Armed Forces of the Philippines (AFP) sai...


In [ ]:
gmaspec = getSpecificArticle(gma_week, keywords=['Sudan', 'Balikatan', 'SIM registration', 'SIM card registration'])
gmaspec.drop_duplicates()
gmaspec

,Text,Annotate
0,Fishermen protest no-sail zone policy amid Bal...,
1,Fishermen in Zambales protested on Wednesday ...,
2,"“Ang buhay kasi ng mangingisda eh isang, kahi...",
3,(The life of a fisherman is hard. The impact ...,
4,De Jesus was also concerned about the effects...,
...,...,...
210,Committee chairperson Senator Imee Marcos sai...,
211,"Aguilar, however, maintained that the selecti...",
212,“Ito ay ayon siyempre sa mga nangyayaring dev...,
213,(This was according to the developments happe...,


In [ ]:
gmaspec.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-spec.csv', index=False)

#### Per Topic

In [ ]:
gmaspec1 = getSpecificArticle(gma_week, keywords=['Sudan'])
gmaspec1.drop_duplicates()
gmaspec1

gmaspec1.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-spec-Sudan.csv', index=False)
gmaspec1

,Text,Annotate
0,DMW: PH gov’t eyeing jobs in Saudi Arabia for ...,
1,The Philippine government is working to find ...,
2,"""We are trying to explore the possibility tha...",
3,"""Then tingnan natin kung baka humupa 'yung si...",
4,"Ople will be leaving for Cairo, Egypt tonight...",
5,She said 725 Filipinos contacted the Philippi...,
6,"""The President's directives were number one, ...",
7,"""To quote the President, kung marunong mag-Ta...",
8,Ople said welfare assistance desks would be e...,
9,"Jinggoy Estrada to DOLE, OWWA: Ensure alternat...",


In [ ]:
gmaspec2 = getSpecificArticle(gma_week, keywords=[ 'Balikatan'])
gmaspec2.drop_duplicates()
gmaspec2

gmaspec2.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-spec-balikatan.csv', index=False)
gmaspec2

,Text,Annotate
0,Fishermen protest no-sail zone policy amid Bal...,
1,Fishermen in Zambales protested on Wednesday ...,
2,"“Ang buhay kasi ng mangingisda eh isang, kahi...",
3,(The life of a fisherman is hard. The impact ...,
4,De Jesus was also concerned about the effects...,
...,...,...
106,Committee chairperson Senator Imee Marcos sai...,
107,"Aguilar, however, maintained that the selecti...",
108,“Ito ay ayon siyempre sa mga nangyayaring dev...,
109,(This was according to the developments happe...,


In [ ]:
gmaspec3 = getSpecificArticle(gma_week, keywords=['SIM registration', 'SIM card registration'])
gmaspec3.drop_duplicates()
gmaspec3

gmaspec3.to_csv(f'{dir}dataset/cleaned_newsArticles/gma-spec-SIM.csv', index=False)
gmaspec3

,Text,Annotate
0,Supreme Court denies TRO bid vs. SIM registration,
1,BAGUIO CITY — The Supreme Court on Tuesday de...,
2,"""The court during its en banc deliberations t...",
3,He said the court instead required the respon...,
4,The SC has yet to disclose the grounds for th...,
...,...,...
80,“I don’t want to posit any opinions on that m...,
81,“But of course our eyes will be on the telcos...,
82,Remulla said he would meet with a cabinet clu...,
83,"Last week, the National Commission said the ...",


In [ ]:
def searchK(df,outlet,key,name,isSunstar=False):
    gmaspec = getSpecificArticle(df, keywords=key, isSunStar=isSunstar)
    gmaspec.drop_duplicates()

    gmaspec.to_csv(f'{dir}dataset/cleaned_newsArticles/topic/{outlet}-spec-{name}.csv', index=False)
    return gmaspec

In [ ]:
searchK(df=gma,outlet='gma',key=['inflation'],name='inflation')

,Text,Annotate
0,"Groups hold protest on Labor Day, call for wag...",
1,Several workers' groups staged protest action...,
2,"“Kilusang Mayo Uno (KMU), together with diffe...",
3,"“Across the country, several protests rallied...",
4,"In the National Capital Region, KMU said Phil...",
...,...,...
396,"""I am confident as to his holistic fitness to...",
397,"Representatives Ramon Guico Jr., Johnny Pimen...",
398,Before approving the motion to confirm Pascua...,
399,"""Mr. Secretary, your department is key to the...",


In [ ]:
searchK(df=gma,outlet='gma',key=['China'],name='china')

,Text,Annotate
0,Marcos: Bilateral defense guidelines with US ‘...,
1,"WASHINGTON — President Ferdinand ""Bongbong"" M...",
2,"""Well, one, that is responsive to the challen...",
3,Department of National Defense officer-in-cha...,
4,"The said guidelines ""reaffirmed that an armed...",
...,...,...
3125,Huang also said he learned from media reports...,
3126,"These moves, Huang said, showed ""concrete imp...",
3127,"""These stories are also good testament to the...",
3128,The PCG said that they received a report at a...,


In [ ]:
searchK(df=gma,outlet='gma',key=['inflation'],name='inflation')

## PNA

In [ ]:
pna = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/PhilippineNewsAgency_2023-05-06NEWS.csv')
pna

,Unnamed: 0,Title,Author,Date,Text
0,0,"MIAA, Meralco begins NAIA T3 electrical audit",NaN,"May 5, 2023",File photo\nMANILA – The Manila International ...
1,1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",NaN,"May 5, 2023","MANILA – Marilene Acosta, the CEO of Pag-IBIG ..."
2,2,DMW assures aid for OFWS from Sudan who wanted...,By Marita Moaje,"May 5, 2023",HOME. A total of 74 repatriates from war-torn ...
3,3,PNP chief fetes HPG's vital role in upholding ...,By Christopher Lloyd Caliwan,"May 5, 2023",PNP chief Gen. Benjamin Acorda Jr. (PNA photo ...
4,4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",By Raymond Carl Dela Cruz,"May 5, 2023",MANILA – The chief of the Land Transportation ...
...,...,...,...,...,...
2195,2195,PBBM's 'Pambansang Pabahay’ empowers LGUs to a...,By Zaldy De Layola,"February 8, 2023",(File photo)\nMANILA – The Department of Human...
2196,2196,1.2K cops to secure ASEAN education execs' mee...,By Christopher Lloyd Caliwan,"February 8, 2023",(Photo courtesy of NCRPO)\nMANILA – More than ...
2197,2197,PNP reaffirms support to Interpol efforts vs. ...,By Christopher Lloyd Caliwan,"February 8, 2023",Interpol Secretary General Jürgen Stock (left)...
2198,2198,"IP leaders call for unity to preserve IPRA, NC...","By Perfecto Raymundo, Jr. and Che Palicte","February 8, 2023",UNITY. Indigenous people leaders call for unit...


In [ ]:
pna['Date'] = pd.to_datetime(pna['Date'])
pna

,Unnamed: 0,Title,Author,Date,Text
0,0,"MIAA, Meralco begins NAIA T3 electrical audit",NaN,2023-05-05,File photo\nMANILA – The Manila International ...
1,1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",NaN,2023-05-05,"MANILA – Marilene Acosta, the CEO of Pag-IBIG ..."
2,2,DMW assures aid for OFWS from Sudan who wanted...,By Marita Moaje,2023-05-05,HOME. A total of 74 repatriates from war-torn ...
3,3,PNP chief fetes HPG's vital role in upholding ...,By Christopher Lloyd Caliwan,2023-05-05,PNP chief Gen. Benjamin Acorda Jr. (PNA photo ...
4,4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",By Raymond Carl Dela Cruz,2023-05-05,MANILA – The chief of the Land Transportation ...
...,...,...,...,...,...
2195,2195,PBBM's 'Pambansang Pabahay’ empowers LGUs to a...,By Zaldy De Layola,2023-02-08,(File photo)\nMANILA – The Department of Human...
2196,2196,1.2K cops to secure ASEAN education execs' mee...,By Christopher Lloyd Caliwan,2023-02-08,(Photo courtesy of NCRPO)\nMANILA – More than ...
2197,2197,PNP reaffirms support to Interpol efforts vs. ...,By Christopher Lloyd Caliwan,2023-02-08,Interpol Secretary General Jürgen Stock (left)...
2198,2198,"IP leaders call for unity to preserve IPRA, NC...","By Perfecto Raymundo, Jr. and Che Palicte",2023-02-08,UNITY. Indigenous people leaders call for unit...


In [ ]:
pna_week =  pna.loc[pna['Date'] >= np.datetime64('2023-04-23')]
pna_week = pna_week.loc[ pna_week['Date'] <= np.datetime64('2023-04-29')]
pna_week = pna_week.reset_index()
pna_week = pna_week.drop(['Unnamed: 0', 'index'], axis=1)
pna_week

,Title,Author,Date,Text
0,First batch of 17 Filipino evacuees from Sudan...,By Joyce Ann L. Rocamora,2023-04-29,SAFE AT LAST. Foreign Affairs Secretary Enriqu...
1,PBBM foreign trips boost PH image as investmen...,NaN,2023-04-29,Philippine Economic Zone Authority Deputy Dire...
2,Contingency plan in place; new core team eyed ...,By Joyce Ann L. Rocamora,2023-04-29,DILG Central Office Disaster Information Coord...
3,Kadiwa offers affordable products to workers o...,By Ferdinand Patinio,2023-04-29,KADIWA NG PANGULO. President Ferdinand R. Marc...
4,DSWD to assist 8 Filipinos arriving from Sudan,NaN,2023-04-29,MANILA – The Department of Social Welfare and ...
...,...,...,...,...
161,PAF personnel briefed on American Patriot miss...,By Priam Nepomuceno,2023-04-23,960th Air and Missile Defense Group personnel ...
162,"PBBM hosts concert for soldiers, families",NaN,2023-04-23,"NIGHT OF MUSIC. Performers at the first ""Konsy..."
163,Marcos-Qin talks productive; ‘some pronounceme...,By Joyce Ann L. Rocamora,2023-04-23,COURTESY CALL. President Ferdinand R. Marcos J...
164,Gov't prepares plans to bring out trapped Pino...,By Joyce Ann L. Rocamora,2023-04-23,MANILA – President Ferdinand R. Marcos Jr. on ...


In [ ]:
pnaspec = getSpecificArticle(pna_week, keywords=['Sudan', 'Balikatan', 'SIM registration', 'SIM card registration'])
pnaspec.drop_duplicates()
pnaspec

,Text,Annotate
0,First batch of 17 Filipino evacuees from Sudan...,
1,"MANILA – Government officials, led by Departme...",
2,The group arrived past 6:30 p.m. at Terminal 3...,
3,Among the evacuees from conflict-torn Sudan ar...,
4,A report from the Department of Social Welfare...,
...,...,...
513,The SAF said in a statement on Facebook that a...,
514,"""We are waiting for the process to begin in h...",
515,It added that the Saudi diplomatic mission was...,
516,Kuwait also said Saturday that it had launched...,


In [ ]:
pnaspec.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec.csv', index=False)

#### Per Topic

In [ ]:
pnaspec1 = getSpecificArticle(pna_week, keywords=['Sudan'])
pnaspec1.drop_duplicates()


pnaspec1.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec-Sudan.csv', index=False)
pnaspec1

,Text,Annotate
0,First batch of 17 Filipino evacuees from Sudan...,
1,"MANILA – Government officials, led by Departme...",
2,The group arrived past 6:30 p.m. at Terminal 3...,
3,Among the evacuees from conflict-torn Sudan ar...,
4,A report from the Department of Social Welfare...,
...,...,...
220,The SAF said in a statement on Facebook that a...,
221,"""We are waiting for the process to begin in h...",
222,It added that the Saudi diplomatic mission was...,
223,Kuwait also said Saturday that it had launched...,


In [ ]:
pnaspec2 = getSpecificArticle(pna_week, keywords=[ 'Balikatan'])
pnaspec2.drop_duplicates()


pnaspec2.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec-balikatan.csv', index=False)
pnaspec2

,Text,Annotate
0,AFP: Balikatan 2023 ‘very successful’,
1,BROTHERHOOD. Filipino and American soldiers bi...,
2,"MANILA – This year’s iteration of the ""Balikat...",
3,"He said this year’s ""Balikatan,"" which lasted ...",
4,"“We had a very successful Balikatan this year,...",
...,...,...
176,"The GBADS contract is worth PHP6,846,750,000 a...",
177,The Department of National Defense signed the ...,
178,"Meanwhile, the PAF's 15th Strike Wing's A-29B ...",
179,"""The exercise was conducted as part of their c...",


In [ ]:
pnaspec3 = getSpecificArticle(pna_week, keywords=['SIM registration', 'SIM card registration'])
pnaspec3.drop_duplicates()


pnaspec3.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec-SIM.csv', index=False)
pnaspec3

,Text,Annotate
0,Romualdez calls for easier SIM registration,
1,MANILA – Speaker Ferdinand Martin G. Romualdez...,
2,"In a statement, Romualdez thanked President Fe...",
3,“Let us help the millions of Filipinos who hav...,
4,"“In particular, let us assist overseas Filipin...",
...,...,...
107,A one-time code or personal identification num...,
108,"""(T)he law is the law. We have to follow the l...",
109,"Over the weekend, Department of Information an...",
110,"""We will identify the real problem of why some...",


In [ ]:
pnaspec4 = getSpecificArticle(pna_week, keywords=['remote learning'])
pnaspec4.drop_duplicates()


pnaspec4.to_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec-DepEd.csv', index=False)
pnaspec4

,Text,Annotate
0,DepEd: Schools may resort to remote learning d...,
1,PNA file photo,
2,MANILA – The Department of Education (DepEd) r...,
3,"In a Viber message, DepEd spokesperson Michael...",
4,“Kaya po mariin naming pinapaalalahan ang atin...,
5,"Under ADM, learners may continue their classes...",
6,The Federation of Parents-Teachers Association...,


## Rappler

In [ ]:
rappler = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/Rappler_2023-05-07NEWS.csv')
rappler

,Unnamed: 0,Title,Author,Date,Text
0,0,Marcos the party-going schoolboy returns to UK...,Lian Buan,2023-05-05,"LONDON, United Kingdom – In the late ’70s, Fer..."
1,1,"Ablan confirms resignation from DepEd, denies ...",Ryan Macasero,2023-05-05,"MANILA, Philippines – Former Department of Edu..."
2,2,WATCH: Paano naibenta sa labas ang DepEd lapto...,Rappler.com,2023-05-05,"MANILA, Philippines – Naaalala ‘nyo ba ‘yung m..."
3,3,"In unprecedented move, Filipino journalist har...",Gaby Baizas,2023-05-05,"MANILA, Philippines – Days after media workers..."
4,4,Marcos says Philippines bases could be ‘useful...,Reuters,2023-05-05,"WASHINGTON DC, USA – Philippines President Fer..."
...,...,...,...,...,...
2196,2196,Fewer Filipinos skeptical of COVID-19 jabs – SWS,Sofia Tomacruz,2022-01-21,"MANILA, Philippines – As the coronavirus is po..."
2197,2197,"Duterte appoints Malacañang official, ex-Napol...",NaN,2022-01-21,"MANILA, Philippines – President Rodrigo Dutert..."
2198,2198,‘Ironclad’ PH-South Korea alliance seen with s...,Jairo Bolledo,2022-01-21,"MANILA, Philippines – The Philippines is looki..."
2199,2199,"3 Cordillera provinces, Northern Samar placed ...",NaN,2022-01-20,"MANILA, Philippines – The alert level of three..."


In [ ]:
rappler['Date'] = pd.to_datetime(rappler['Date'])
rappler

,Unnamed: 0,Title,Author,Date,Text
0,0,Marcos the party-going schoolboy returns to UK...,Lian Buan,2023-05-05,"LONDON, United Kingdom – In the late ’70s, Fer..."
1,1,"Ablan confirms resignation from DepEd, denies ...",Ryan Macasero,2023-05-05,"MANILA, Philippines – Former Department of Edu..."
2,2,WATCH: Paano naibenta sa labas ang DepEd lapto...,Rappler.com,2023-05-05,"MANILA, Philippines – Naaalala ‘nyo ba ‘yung m..."
3,3,"In unprecedented move, Filipino journalist har...",Gaby Baizas,2023-05-05,"MANILA, Philippines – Days after media workers..."
4,4,Marcos says Philippines bases could be ‘useful...,Reuters,2023-05-05,"WASHINGTON DC, USA – Philippines President Fer..."
...,...,...,...,...,...
2196,2196,Fewer Filipinos skeptical of COVID-19 jabs – SWS,Sofia Tomacruz,2022-01-21,"MANILA, Philippines – As the coronavirus is po..."
2197,2197,"Duterte appoints Malacañang official, ex-Napol...",NaN,2022-01-21,"MANILA, Philippines – President Rodrigo Dutert..."
2198,2198,‘Ironclad’ PH-South Korea alliance seen with s...,Jairo Bolledo,2022-01-21,"MANILA, Philippines – The Philippines is looki..."
2199,2199,"3 Cordillera provinces, Northern Samar placed ...",NaN,2022-01-20,"MANILA, Philippines – The alert level of three..."


In [ ]:
rappler_week =  rappler.loc[rappler['Date'] >= np.datetime64('2023-04-23')]
rappler_week = rappler_week.loc[ rappler_week['Date'] <= np.datetime64('2023-04-29')]
rappler_week = rappler_week.reset_index()
rappler_week = rappler_week.drop(['Unnamed: 0', 'index'], axis=1)
rappler_week

,Title,Author,Date,Text
0,First batch of Filipino evacuees from Sudan re...,Jairo Bolledo,2023-04-29,"MANILA, Philippines – The first batch of Filip..."
1,"US military: PH owns EDCA sites, can only be u...",Jairo Bolledo,2023-04-29,"MANILA, Philippines – A United States military..."
2,Merceditas Gutierrez joins GSIS board,Bea Cupin,2023-04-28,"MANILA, Philippines – Merceditas Gutierrez, th..."
3,‘A no-brainer’: Marcos gov’t urged to declare ...,Isagani de Castro Jr.,2023-04-28,"MANILA, Philippines – The Marcos administratio..."
4,"As regional tensions rise, PH eyes ‘bigger’ Ba...",Ryan Macasero,2023-04-28,"MANILA, Philippines – No further proof was nee..."
5,"Chinese, Philippine vessels almost crash near ...",Ryan Macasero,2023-04-28,"MANILA, Philippines – A Chinese Coast Guard sh..."
6,Supreme Court affirms: Manila Doctors Hospital...,Jairo Bolledo,2023-04-28,"MANILA, Philippines – The Supreme Court’s (SC)..."
7,"LIVESTREAM: Philippines, US close 2023 Balikat...",Rappler.com,2023-04-28,"MANILA, Philippines – The largest Balikatan (S..."
8,"[WATCH] Philippine, US troops flex interoperab...",Rappler.com,2023-04-27,"MANILA, Philippines — The largest Balikatan wr..."
9,Muntinlupa court issues hold departure order v...,Jairo Bolledo,2023-04-27,"MANILA, Philippines – Following the issuance o..."


In [ ]:
rapplerspec = getSpecificArticle(rappler_week, keywords=['Sudan', 'Balikatan', 'SIM registration', 'SIM card registration'])
rapplerspec.drop_duplicates()
rapplerspec

,Text,Annotate
0,First batch of Filipino evacuees from Sudan re...,
1,"MANILA, Philippines – The first batch of Filip...",
2,The 17 Filipino evacuees from the African coun...,
3,"According to the DFA, eight of the rescued Fil...",
4,"As I the 1st batch of repatriates from Sudan,...",
...,...,...
125,"Under Marcos, Americans were granted access to...",
126,Chinese Ambassador to Manila Huang Xillian adv...,
127,Marcos is flying to the US days after Balikata...,
128,"Aside from the bilateral meeting with Biden, M...",


In [ ]:
rapplerspec.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec.csv', index=False)

#### Per Topic

In [ ]:
rapplerspec1 = getSpecificArticle(rappler_week, keywords=['Sudan'])
rapplerspec1.drop_duplicates()


rapplerspec1.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec-Sudan.csv', index=False)
rapplerspec1

,Text,Annotate
0,First batch of Filipino evacuees from Sudan re...,
1,"MANILA, Philippines – The first batch of Filip...",
2,The 17 Filipino evacuees from the African coun...,
3,"According to the DFA, eight of the rescued Fil...",
4,"As I the 1st batch of repatriates from Sudan,...",
5,"Meanwhile, the nine other Filipinos, including...",
6,The DFA said a total of 610 Filipinos had left...,
7,The foreign affairs department said another 45...,
8,"On April 15, Sudan’s paramilitary group, Rapid...",
9,The World Health Organization said the conflic...,


In [ ]:
rapplerspec2 = getSpecificArticle(rappler_week, keywords=[ 'Balikatan'])
rapplerspec2.drop_duplicates()


rapplerspec2.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec-balikatan.csv', index=False)
rapplerspec2

,Text,Annotate
0,"US military: PH owns EDCA sites, can only be u...",
1,"MANILA, Philippines – A United States military...",
2,During his meeting with Manila’s acting Depart...,
3,"On February 2, Galvez and US Defense Secretary...",
4,The agreement on the establishment of EDCA sit...,
...,...,...
103,"Under Marcos, Americans were granted access to...",
104,Chinese Ambassador to Manila Huang Xillian adv...,
105,Marcos is flying to the US days after Balikata...,
106,"Aside from the bilateral meeting with Biden, M...",


In [ ]:
rapplerspec3 = getSpecificArticle(rappler_week, keywords=['SIM registration', 'SIM card registration'])
rapplerspec3.drop_duplicates()


rapplerspec3.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec-SIM.csv', index=False)
rapplerspec3

,Text,Annotate
0,SC junks request to temporarily stop SIM card ...,
1,"BAGUIO CITY, Philippines – The Supreme Court (...",
2,High Court spokesperson lawyer Brian Keith Hos...,
3,"“The Supreme Court, during its En Banc deliber...",
4,"On Tuesday morning, Department of Justice (DOJ...",
5,Remulla later told reporters on Tuesday that D...,
6,"On October 10, 2022, President Ferdinand Marco...",
7,"Since the law’s introduction, many groups have...",
8,The High Court’s latest action stemmed from th...,
9,"Aside from this, the groups also asked the SC ...",


In [ ]:
rapplerspec4 = getSpecificArticle(rappler_week, keywords=['remote learning'])
rapplerspec4.drop_duplicates()


rapplerspec4.to_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec-DepEd.csv', index=False)
rapplerspec4

,Text,Annotate
0,Schools may shift to remote learning during ex...,
1,"MANILA, Philippines – The Department of Educat...",
2,“The principals/school heads have been reminde...,
3,Poa said that this was already contained in a ...,
4,“Iba-iba po kasi ang situation ng ating mga pa...,
5,(Each school has a different situation. That’s...,
6,Schools in the country have reverted to in-per...,
7,Weather bureau PAGASA had warned that the coun...,
8,"Senator Sherwin Gatchalian, chairman on basic...",


## Sunstar


In [ ]:
sunstar = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/SunStar_2023-05-07NEWS.csv')
sunstar

,Unnamed: 0,Title,Author,Date,Text
0,0,Sandiganbayan affirms denial of motion seeking...,Third Anne Peralta-Malonzo,"May 07, 2023",THE Sandiganbayan has affirmed the decision of...
1,1,PH to reassess protocols with end of Covid glo...,NaN,"May 06, 2023",WITH the end of the global emergency status fo...
2,2,DMW: Work experience top criterion for oversea...,NaN,"May 06, 2023",THE Department of Migrant Workers (DMW) on Sat...
3,3,Marcos accepts courtesy resignation of 2 senio...,Third Anne Peralta-Malonzo,"May 05, 2023",PRESIDENT Ferdinand “Bongbong” Marcos has conf...
4,4,"DOH reiterates call for kids to get measles, p...",NaN,"May 05, 2023",FIVE days after starting its mass immunization...
...,...,...,...,...,...
2261,2261,"CSF indigents get cash aid, food packs from Guv",Ian Ocampo Flora,"January 30, 2023",CITY OF SAN FERNANDO The provincial government...
2262,2262,AC execs help Bale Pusu ward reunite with family,Charlene A. Cayabyab,"January 28, 2023",ANGELES CITY The city government here has faci...
2263,2263,Cops seize P14-M 'shabu' in Bulacan,Princess Clea Arcellaz,"January 28, 2023",CITY OF SAN FERNANDO More than two kilos of su...
2264,2264,Crissot's 156 birth anniversary commemorated,Ian Ocampo Flora,"January 28, 2023",BACOLOR The heirs of Juan Crisostomo Soto (Cri...


In [ ]:
sunstar['Date'] = pd.to_datetime(sunstar['Date'])
sunstar

,Unnamed: 0,Title,Author,Date,Text
0,0,Sandiganbayan affirms denial of motion seeking...,Third Anne Peralta-Malonzo,2023-05-07,THE Sandiganbayan has affirmed the decision of...
1,1,PH to reassess protocols with end of Covid glo...,NaN,2023-05-06,WITH the end of the global emergency status fo...
2,2,DMW: Work experience top criterion for oversea...,NaN,2023-05-06,THE Department of Migrant Workers (DMW) on Sat...
3,3,Marcos accepts courtesy resignation of 2 senio...,Third Anne Peralta-Malonzo,2023-05-05,PRESIDENT Ferdinand “Bongbong” Marcos has conf...
4,4,"DOH reiterates call for kids to get measles, p...",NaN,2023-05-05,FIVE days after starting its mass immunization...
...,...,...,...,...,...
2261,2261,"CSF indigents get cash aid, food packs from Guv",Ian Ocampo Flora,2023-01-30,CITY OF SAN FERNANDO The provincial government...
2262,2262,AC execs help Bale Pusu ward reunite with family,Charlene A. Cayabyab,2023-01-28,ANGELES CITY The city government here has faci...
2263,2263,Cops seize P14-M 'shabu' in Bulacan,Princess Clea Arcellaz,2023-01-28,CITY OF SAN FERNANDO More than two kilos of su...
2264,2264,Crissot's 156 birth anniversary commemorated,Ian Ocampo Flora,2023-01-28,BACOLOR The heirs of Juan Crisostomo Soto (Cri...


In [ ]:
sunstar_week =  sunstar.loc[sunstar['Date'] >= np.datetime64('2023-04-23')]
sunstar_week = sunstar_week.loc[ sunstar_week['Date'] <= np.datetime64('2023-04-29')]
sunstar_week = sunstar_week.reset_index()
sunstar_week = sunstar_week.drop(['Unnamed: 0', 'index'], axis=1)
sunstar_week

,Title,Author,Date,Text
0,Migrante calls for justice for 4 OFWs killed i...,NaN,2023-04-29,AN ORGANIZATION of overseas Filipino workers (...
1,"30,000 dengue cases reported in PH for Q1 2023",NaN,2023-04-29,DENGUE cases in the country have breached the ...
2,PDEA destroys P4B dangerous drugs in Cavite,Laureen Mondonedo-Ynot,2023-04-28,"AROUND 700 kilograms, or P4 billion worth of d..."
3,PH detects additional 130 Omicron subvariant c...,NaN,2023-04-28,THE Philippines has detected an additional 130...
4,DMW to charter flight for repatriation of 340 ...,NaN,2023-04-28,THREE hundred forty overseas Filipino workers ...
...,...,...,...,...
185,"Agri-caravan earns P839, 460",Ian Ocampo Flora,2023-04-24,CITY OF SAN FERNANDO The second Kadiwa ng Pan...
186,6 nabbed in cops' drug buy-bust,Princess Clea Arcellaz,2023-04-24,CITY OF SAN FERNANDO Six persons were arrested...
187,"School heads may suspend classes due to heat, ...",Jovi De Leon,2023-04-24,CITY OF SAN FERNANDO The Department of Educati...
188,Pampanga COVID cases down to 55,Ian Ocampo Flora,2023-04-24,CITY OF SAN FERNANDO Pampanga’s active coronav...


In [ ]:
sunstarspec = getSpecificArticle(sunstar_week, isSunStar=True, keywords=['Sudan', 'Balikatan', 'SIM registration', 'SIM card registration'])
sunstarspec.drop_duplicates()
sunstarspec

,Text,Annotate
0,DMW to charter flight for repatriation of 340 ...,
1,THREE hundred forty overseas Filipino workers ...,
2,"In an online press briefing, DMW Secretary Sus...",
3,"""We are now exploring the possibility of a cha...",
4,So certain arrangements will have to be made b...,
...,...,...
441,"""After the April 26 deadline, subscribers will...",
442,Most of the services will be cut off with the ...,
443,"So, there will be a social media unavailabilit...",
444,DICT Secretary Ivan John Uy said that if the 1...,


In [ ]:
sunstarspec.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec.csv', index=False)

#### Per Topic

In [ ]:
sunstarspec1 = getSpecificArticle(sunstar_week, isSunStar=True, keywords=['Sudan'])
sunstarspec1.drop_duplicates()


sunstarspec1.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec-Sudan.csv', index=False)
sunstarspec1

,Text,Annotate
0,DMW to charter flight for repatriation of 340 ...,
1,THREE hundred forty overseas Filipino workers ...,
2,"In an online press briefing, DMW Secretary Sus...",
3,"""We are now exploring the possibility of a cha...",
4,So certain arrangements will have to be made b...,
5,She said the chartered flight in handy in has...,
6,"""They will have to stay longer in Cairo (Egypt...",
7,The 340 OFWs are part of the Filipinos current...,
8,"Once they are fully accounted for, Ople said t...",
9,"""It has to be the choice of the workers.",


In [ ]:
sunstarspec2 = getSpecificArticle(sunstar_week,isSunStar=True, keywords=[ 'Balikatan'])
sunstarspec2.drop_duplicates()


sunstarspec2.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec-balikatan.csv', index=False)
sunstarspec2

,Text,Annotate
0,Marcos witnesses live-fire exercise in Balikat...,
1,PRESIDENT Ferdinand “Bongbong” Marcos Jr. witn...,
2,Marcos was joined by Defense Secretary Carlito...,
3,US Ambassador to the Philippines MaryKay Carls...,
4,"Some 1,400 marines, soldiers, sailors, airmen ...",
5,The exercise included the sinking of the targe...,
6,It was positioned 12 nautical miles from the s...,
7,The vessel was treated as hostile and needed t...,
8,"During the littoral live-fire event, a US Mari...",
9,Among the weapons used were US and Philippine ...,


In [ ]:
sunstarspec3 = getSpecificArticle(sunstar_week, isSunStar=True, keywords=['SIM registration', 'SIM card registration'])
sunstarspec3.drop_duplicates()


sunstarspec3.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec-SIM.csv', index=False)
sunstarspec3

,Text,Annotate
0,NTC: 90-day extension for SIM registration ‘fi...,
1,AN OFFICIAL from the National Commission (NTC...,
2,"That will be the final extension po, ‘yung 90-...",
3,Marcos approved the 90-day extension for SIM r...,
4,"Under the SIM Registration Act, which was sign...",
...,...,...
357,"""After the April 26 deadline, subscribers will...",
358,Most of the services will be cut off with the ...,
359,"So, there will be a social media unavailabilit...",
360,DICT Secretary Ivan John Uy said that if the 1...,


In [ ]:
sunstarspec4 = getSpecificArticle(sunstar_week, keywords=['DepEd reminds'])
sunstarspec4.drop_duplicates()


sunstarspec4.to_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec-DepEd.csv', index=False)
sunstarspec4

,Text,Annotate
0,DepEd reminds schools of rules for suspending ...,


## Manila Bulletin


In [ ]:
mb = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/manilaBulletin_2023-05-05_NEWS.csv')
mb

,Unnamed: 0,Title,Author,Date,Text
0,0,'Thank you gov': Cong Roman mourns passing of ...,Ellson Quismorio,"May 5, 2023",Bataan 1st district Rep. Geraldine Roman paid ...
1,1,P5-M reward offered to find missing Chinese pa...,Martin Sadongdong,"May 5, 2023",A P5-million reward was offered to anyone who ...
2,2,"Hontiveros: Nuclear energy a terrible, false s...",Hannah Torregoza,"May 5, 2023","\nSenator Risa Hontiveros on Friday, May 5 qu..."
3,3,You go girl! Pro-sports solon hails Kaila Napo...,Ellson Quismorio,"May 5, 2023",The first one is always special.\nPuwersa ng B...
4,4,Expedite release of Excellence in Teacher Educ...,Merlina Hernando-Malipot,"May 5, 2023",To further strengthen teacher education in the...
...,...,...,...,...,...
1999,1999,"Diokno, Australian envoy share notes on deepen...",Raymund Antonio,"Mar 25, 2023",Finance Secretary Benjamin Diokno met with Aus...
2000,2000,Australian gov't drums up interest in PH airpo...,Raymund Antonio,"Mar 25, 2023",The Australian government is looking to “drum ...
2001,2001,Proposed EBET law passed on 3rd reading; here'...,Ellson Quismorio,"Mar 25, 2023",The lower chamber recently gave its final appr...
2002,2002,Immediate expansion of gov’t subsidies to kind...,Merlina Hernando-Malipot,"Mar 25, 2023",A network of advocates called for the immediat...


In [ ]:
mb['Date'] = pd.to_datetime(mb['Date'])
mb

,Unnamed: 0,Title,Author,Date,Text
0,0,'Thank you gov': Cong Roman mourns passing of ...,Ellson Quismorio,2023-05-05,Bataan 1st district Rep. Geraldine Roman paid ...
1,1,P5-M reward offered to find missing Chinese pa...,Martin Sadongdong,2023-05-05,A P5-million reward was offered to anyone who ...
2,2,"Hontiveros: Nuclear energy a terrible, false s...",Hannah Torregoza,2023-05-05,"\nSenator Risa Hontiveros on Friday, May 5 qu..."
3,3,You go girl! Pro-sports solon hails Kaila Napo...,Ellson Quismorio,2023-05-05,The first one is always special.\nPuwersa ng B...
4,4,Expedite release of Excellence in Teacher Educ...,Merlina Hernando-Malipot,2023-05-05,To further strengthen teacher education in the...
...,...,...,...,...,...
1999,1999,"Diokno, Australian envoy share notes on deepen...",Raymund Antonio,2023-03-25,Finance Secretary Benjamin Diokno met with Aus...
2000,2000,Australian gov't drums up interest in PH airpo...,Raymund Antonio,2023-03-25,The Australian government is looking to “drum ...
2001,2001,Proposed EBET law passed on 3rd reading; here'...,Ellson Quismorio,2023-03-25,The lower chamber recently gave its final appr...
2002,2002,Immediate expansion of gov’t subsidies to kind...,Merlina Hernando-Malipot,2023-03-25,A network of advocates called for the immediat...


In [ ]:
mb_week =  mb.loc[mb['Date'] >= np.datetime64('2023-04-23')]
mb_week = mb_week.loc[ mb_week['Date'] <= np.datetime64('2023-04-29')]
mb_week = mb_week.reset_index()
mb_week = mb_week.drop(['Unnamed: 0', 'index'], axis=1)
mb_week

,Title,Author,Date,Text
0,No lotto jackpot winners in April 29 evening draw,Luisa K. Cabato,2023-04-29,There were no jackpot winners for the Philippi...
1,F​ilipino evacuees from Sudan start arriving i...,Joseph Pedrajas,2023-04-29,The first batch of Filipinos who fled from Sud...
2,PH sees 304 additional Covid-19 cases,Analou de Vera,2023-04-29,The Philippines registered 304 additional case...
3,PAGASA: Heat index in Cagayan hits 43°C,Charie Mae F. Abarca,2023-04-29,The state weather bureau said the highest comp...
4,LTO not allowed to use earnings to buy plastic...,Aaron Recuenco,2023-04-29,The Land Transportation Office (LTO) is not au...
...,...,...,...,...
357,Cong Vargas earns bragging rights in terms of ...,Ellson Quismorio,2023-04-23,Neophyte solon Quezon 5th district Rep. PM Var...
358,"Expect partly cloudy weather, isolated rain sh...",Ellalyn De Vera-Ruiz,2023-04-23,"The Philippine Atmospheric, Geophysical and As..."
359,'Alagang Tingog Centers' continue to mushroom ...,Ellson Quismorio,2023-04-23,A total of five Alagang Tingog Centers (ATC) h...
360,Marcos: Gov't preparing contingencies to ensur...,Argyll Cyrus Geducos,2023-04-23,"President Ferdinand ""Bongbong"" Marcos Jr. assu..."


In [ ]:
mbspec = getSpecificArticle(mb_week, keywords=['Sudan', 'Balikatan', 'SIM registration', 'SIM card registration'])
mbspec.drop_duplicates()
mbspec

,Text,Annotate
0,F​ilipino evacuees from Sudan start arriving i...,
1,The first batch of Filipinos who fled from Sud...,
2,Department of Foreign Affairs (DFA) Secretary ...,
3,"Nine of them, including an infant, got out of ...",
4,The Philippine consulate received them on Apri...,
...,...,...
749,The President said the government was working ...,
750,"""We are just waiting to get better information...",
751,The in-fighting between the Sudanese Armed For...,
752,According to the World Health Organization (WH...,


In [ ]:
mbspec.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec.csv', index=False)

#### Per Topic

In [ ]:
mbspec1 = getSpecificArticle(mb_week, keywords=['Sudan'])
mbspec1.drop_duplicates()


mbspec1.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec-Sudan.csv', index=False)
mbspec1

,Text,Annotate
0,F​ilipino evacuees from Sudan start arriving i...,
1,The first batch of Filipinos who fled from Sud...,
2,Department of Foreign Affairs (DFA) Secretary ...,
3,"Nine of them, including an infant, got out of ...",
4,The Philippine consulate received them on Apri...,
...,...,...
265,The President said the government was working ...,
266,"""We are just waiting to get better information...",
267,The in-fighting between the Sudanese Armed For...,
268,According to the World Health Organization (WH...,


In [ ]:
mbspec2 = getSpecificArticle(mb_week, keywords=[ 'Balikatan'])
mbspec2.drop_duplicates()


mbspec2.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec-balikatan.csv', index=False)
mbspec2

,Text,Annotate
0,"PH, US to discuss planned joint naval sails in...",
1,The Philippines and the United States have agr...,
2,It was DND Officer in Charge (OIC) Senior Unde...,
3,“Galvez suggested the finalization of joint sa...,
4,"In response, Andolong stated that Aquilino exp...",
...,...,...
131,(Participating soldiers alight a Black Hawk he...,
132,"Meanwhile, PA soldiers and their counterparts ...",
133,Eighty-one soldiers from the PA’s 99th Infantr...,
134,"During an air assault operation, ground-based ...",


In [ ]:
mbspec3 = getSpecificArticle(mb_week, keywords=['SIM registration', 'SIM card registration'])
mbspec3.drop_duplicates()


mbspec3.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec-SIM.csv', index=False)
mbspec3

,Text,Annotate
0,Speaker asks telcos: Reveal total of postpaid ...,
1,House Speaker Martin Romualdez is asking the c...,
2,"data should give us, the public, an idea of h...",
3,He added that telcos should filter their data ...,
4,Romualdez asked for these figures just days af...,
...,...,...
343,"Villafuerte, the majority leader of the powerf...",
344,The SIM registration turnout is reportedly und...,
345,"During Holy Week, the Bicol solon backed the s...",
346,"But the DICT later turned down such appeals, s...",


In [ ]:
mbspec4 = getSpecificArticle(mb_week, keywords=['schools amid extreme heat'])
mbspec4.drop_duplicates()


mbspec4.to_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec-DepEd.csv', index=False)
mbspec4

,Text,Annotate
0,Blended learning being implemented by some sch...,
1,The Department of Education (DepEd) on Wednesd...,
2,“Some schools are already implemented blended ...,
3,“Classes are held at certain times of the day ...,
4,"On April 20, DepEd issued a reminder to school...",
5,The agency also reiterated that the DepEd Orde...,
6,Asked how many schools have temporarily shifte...,
7,Shortened class hours,
8,"Meanwhile, Poa reiterated that DepEd regional ...",
9,"Last week, the Teachers’ Dignity Coalition (TD...",
